- Coding a neural network using only matrix multiplication
- Writting a dataloader to be used in training
- Creating a NN with high level classes and sequential api
- Using an optimizer and pre-defined loss
- Using tf and keras

Data: 
- [mnist_train](https://drive.google.com/file/d/1bCVtZBPQcEz3AqkvrI1M69D-mcY6f5TK/view?usp=sharing)
- [mnist_test](https://drive.google.com/file/d/1DrN5-afU-U6x5hMrUgpUaOA7wYZzXkmz/view?usp=sharing)


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [2]:
local_path_mnist_train = "/Users/gunnvantsaini/OneDrive/project_codes/content/dl_basics/vision/sony/data/mnist_train.csv"
local_path_mnist_test = "/Users/gunnvantsaini/OneDrive/project_codes/content/dl_basics/vision/sony/data/mnist_test.csv"

### Coding a neural network using only matrix multiplication

In [3]:
mnist_train = pd.read_csv(local_path_mnist_train)
mnist_test = pd.read_csv(local_path_mnist_test)

In [4]:
mnist_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X=mnist_train.drop('label',axis=1).values/255.0
y=mnist_train['label'].values
X=torch.tensor(X,dtype=torch.float)
y=torch.tensor(y,dtype=torch.float)

In [6]:
w1=torch.randn((784,3),dtype=torch.float)
b1=torch.randn((3,),dtype=torch.float)
w2=torch.randn((3,10),dtype=torch.float)
b2=torch.randn((10,),dtype=torch.float)

In [7]:
def network(X,w1,b1,w2,b2):
    z1=torch.matmul(X.float(),w1)+b1
    res1=torch.sigmoid(z1)
    z2=torch.matmul(res1,w2)+b2
    probs=torch.softmax(z2,axis=1)
    return probs
def CE(probs,y):
    return -torch.log(probs[range(y.shape[0]),y.long()]).mean()

In [8]:
## Forward pass
p=network(X,w1,b1,w2,b2)

In [9]:
## Loss
CE(p,y)

tensor(3.9186)

In [10]:
## training loop
w1=torch.randn((784,3),dtype=torch.float,requires_grad=True)
b1=torch.randn((3,),dtype=torch.float,requires_grad=True)
w2=torch.randn((3,10),dtype=torch.float,requires_grad=True)
b2=torch.randn((10,),dtype=torch.float,requires_grad=True)
lr=0.1
Loss=[]
for i in range(10):
    p=network(X,w1,b1,w2,b2)
    #print(p)
    loss=CE(p,y)
    loss.backward()
    Loss.append(loss.item())
    acc=(p.argmax(axis=1)==y).float().mean().item()
    print(f"Iter {i+1}, loss {loss.item()}, acc {acc}")
    with torch.no_grad():
        w1-=lr*w1.grad
        b1-=lr*b1.grad
        w2-=lr*w2.grad
        b2-=lr*b2.grad
        w1.grad.zero_()
        b1.grad.zero_()
        w2.grad.zero_()
        b2.grad.zero_()   

Iter 1, loss 3.1811439990997314, acc 0.1008809506893158
Iter 2, loss 3.1651079654693604, acc 0.09964285790920258
Iter 3, loss 3.1496219635009766, acc 0.09883332997560501
Iter 4, loss 3.134650230407715, acc 0.0984523817896843
Iter 5, loss 3.1201581954956055, acc 0.09823809564113617
Iter 6, loss 3.1061158180236816, acc 0.09811905026435852
Iter 7, loss 3.0924930572509766, acc 0.0979047641158104
Iter 8, loss 3.0792653560638428, acc 0.09761904925107956
Iter 9, loss 3.066408157348633, acc 0.09702380746603012
Iter 10, loss 3.0538992881774902, acc 0.09690476208925247


In [11]:
class MnistData(Dataset):
    def __init__(self,X,y):
        self.X=X
        self.y=y
    def __len__(self):
        return X.shape[0]
    def __getitem__(self,idx):
        X=self.X[idx,]
        y=self.y[idx]
        sample={'X':X,'y':y}
        return sample

In [12]:
X=mnist_train.drop('label',axis=1).values/255.0
y=mnist_train['label'].values
mnist=MnistData(X,y)

In [13]:
mnist_batched=DataLoader(mnist,100)

In [14]:
next(iter(mnist_batched))

{'X': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64),
 'y': tensor([1, 0, 1, 4, 0, 0, 7, 3, 5, 3, 8, 9, 1, 3, 3, 1, 2, 0, 7, 5, 8, 6, 2, 0,
         2, 3, 6, 9, 9, 7, 8, 9, 4, 9, 2, 1, 3, 1, 1, 4, 9, 1, 4, 4, 2, 6, 3, 7,
         7, 4, 7, 5, 1, 9, 0, 2, 2, 3, 9, 1, 1, 1, 5, 0, 6, 3, 4, 8, 1, 0, 3, 9,
         6, 2, 6, 4, 7, 1, 4, 1, 5, 4, 8, 9, 2, 9, 9, 8, 9, 6, 3, 6, 4, 6, 2, 9,
         1, 2, 0, 5])}

In [15]:
### Class Excercise use the regression.csv and create a Dataloader using that data

In [16]:
class RegData(Dataset):
    def __init__(self,path,y_name):
        self.path = path
        self.y_name = y_name
        self.data = pd.read_csv(path)
        self.X = self.data.drop(self.y_name,axis=1).values
        self.y = self.data[self.y_name].values
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self,idx):
        x = self.X[idx]
        y = self.y[idx]
        return {'X':x,'y':y}    

In [17]:
d = RegData(path="./data/regression.csv",y_name='mpg')

In [18]:
d_train = DataLoader(d,batch_size=32)

In [19]:
next(iter(d_train))

{'X': tensor([[8.0000e+00, 3.0700e+02, 1.3000e+02, 3.5040e+03, 1.2000e+01, 7.0000e+01,
          1.0000e+00],
         [8.0000e+00, 3.5000e+02, 1.6500e+02, 3.6930e+03, 1.1500e+01, 7.0000e+01,
          1.0000e+00],
         [8.0000e+00, 3.1800e+02, 1.5000e+02, 3.4360e+03, 1.1000e+01, 7.0000e+01,
          1.0000e+00],
         [8.0000e+00, 3.0400e+02, 1.5000e+02, 3.4330e+03, 1.2000e+01, 7.0000e+01,
          1.0000e+00],
         [8.0000e+00, 3.0200e+02, 1.4000e+02, 3.4490e+03, 1.0500e+01, 7.0000e+01,
          1.0000e+00],
         [8.0000e+00, 4.2900e+02, 1.9800e+02, 4.3410e+03, 1.0000e+01, 7.0000e+01,
          1.0000e+00],
         [8.0000e+00, 4.5400e+02, 2.2000e+02, 4.3540e+03, 9.0000e+00, 7.0000e+01,
          1.0000e+00],
         [8.0000e+00, 4.4000e+02, 2.1500e+02, 4.3120e+03, 8.5000e+00, 7.0000e+01,
          1.0000e+00],
         [8.0000e+00, 4.5500e+02, 2.2500e+02, 4.4250e+03, 1.0000e+01, 7.0000e+01,
          1.0000e+00],
         [8.0000e+00, 3.9000e+02, 1.9000e+02, 3.85

### Training using a dataloader

In [20]:
w1=torch.randn((784,3),dtype=torch.float,requires_grad=True)
b1=torch.randn((3,),dtype=torch.float,requires_grad=True)
w2=torch.randn((3,10),dtype=torch.float,requires_grad=True)
b2=torch.randn((10,),dtype=torch.float,requires_grad=True)
lr=0.01
Loss=[]
num_epoch=10
for i in range(num_epoch):
    for j,batch in enumerate(mnist_batched):
        x=batch['X']
        Y=batch['y']
        p=network(x,w1,b1,w2,b2)
        loss=CE(p,Y)
        loss.backward()
        Loss.append(loss.item())
        acc=(p.argmax(axis=1)==Y).float().mean().item()
        if j%100 == 0:
            print(f"Epoch {i+1}, iter {j+1}, loss {loss.item()}, acc {acc}")
        with torch.no_grad():
            w1-=lr*w1.grad
            b1-=lr*b1.grad
            w2-=lr*w2.grad
            b2-=lr*b2.grad
            w1.grad.zero_()
            b1.grad.zero_()
            w2.grad.zero_()
            b2.grad.zero_()   

Epoch 1, iter 1, loss 2.9227001667022705, acc 0.05000000074505806
Epoch 1, iter 101, loss 2.6517159938812256, acc 0.05999999865889549
Epoch 1, iter 201, loss 2.699867010116577, acc 0.17000000178813934
Epoch 1, iter 301, loss 2.7328686714172363, acc 0.07000000029802322
Epoch 1, iter 401, loss 2.6684155464172363, acc 0.029999999329447746
Epoch 2, iter 1, loss 2.491816759109497, acc 0.07999999821186066
Epoch 2, iter 101, loss 2.370453119277954, acc 0.10000000149011612
Epoch 2, iter 201, loss 2.4859681129455566, acc 0.17000000178813934
Epoch 2, iter 301, loss 2.49375581741333, acc 0.05999999865889549
Epoch 2, iter 401, loss 2.4474682807922363, acc 0.03999999910593033
Epoch 3, iter 1, loss 2.326808214187622, acc 0.09000000357627869
Epoch 3, iter 101, loss 2.270258903503418, acc 0.15000000596046448
Epoch 3, iter 201, loss 2.369637966156006, acc 0.10000000149011612
Epoch 3, iter 301, loss 2.3730409145355225, acc 0.10000000149011612
Epoch 3, iter 401, loss 2.3355281352996826, acc 0.14000000059

### Writting a network with a high level api

In [21]:
from torch import nn

In [22]:
#### Create a model with nn class ####
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.w1=nn.Parameter(torch.randn((784,3),dtype=torch.float))
        self.b1=nn.Parameter(torch.randn((3,),dtype=torch.float))
        self.w2=nn.Parameter(torch.randn((3,10),dtype=torch.float))
        self.b2=nn.Parameter(torch.randn((10,),dtype=torch.float))
    def forward(self,X):
        z1=torch.matmul(X.float(),self.w1)+self.b1
        res1=torch.sigmoid(z1)
        z2=torch.matmul(res1,self.w2)+self.b2
        probs=torch.softmax(z2,axis=1)
        return probs

In [23]:
mod=MLP()

In [24]:
lr=0.01
Loss=[]
num_epoch=10
for i in range(num_epoch):
    for j,batch in enumerate(mnist_batched):
        x=batch['X']
        Y=batch['y']
        p=mod(x)
        loss=CE(p,Y)
        loss.backward()
        Loss.append(loss.item())
        acc=(p.argmax(axis=1)==Y).float().mean().item()
        if j%100 == 0:
            print(f"Epoch {i+1}, iter {j+1}, loss {loss.item()}, acc {acc}")
        with torch.no_grad():
            for p in mod.parameters():
                p-=lr*p.grad
            mod.zero_grad()

Epoch 1, iter 1, loss 3.1765024662017822, acc 0.009999999776482582
Epoch 1, iter 101, loss 2.939328670501709, acc 0.03999999910593033
Epoch 1, iter 201, loss 2.653698682785034, acc 0.07999999821186066
Epoch 1, iter 301, loss 2.6578798294067383, acc 0.03999999910593033
Epoch 1, iter 401, loss 2.7457642555236816, acc 0.10999999940395355
Epoch 2, iter 1, loss 2.59014892578125, acc 0.09000000357627869
Epoch 2, iter 101, loss 2.5368409156799316, acc 0.07999999821186066
Epoch 2, iter 201, loss 2.3850996494293213, acc 0.10999999940395355
Epoch 2, iter 301, loss 2.4148108959198, acc 0.07999999821186066
Epoch 2, iter 401, loss 2.4714813232421875, acc 0.14000000059604645
Epoch 3, iter 1, loss 2.3903872966766357, acc 0.09000000357627869
Epoch 3, iter 101, loss 2.3906188011169434, acc 0.10999999940395355
Epoch 3, iter 201, loss 2.2657999992370605, acc 0.10999999940395355
Epoch 3, iter 301, loss 2.3142194747924805, acc 0.10000000149011612
Epoch 3, iter 401, loss 2.3267226219177246, acc 0.1500000059

In [25]:
### Can we improve this further, should we be declaring parameters? Shouldn't there be abstractions for layers?
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1=nn.Linear(784,3)
        self.sig=nn.Sigmoid()
        self.lin2=nn.Linear(3,10)
        self.softmax=nn.Softmax()
    def forward(self,X):
        x=self.lin1(X)
        x=self.sig(x)
        x=self.lin2(x)
        x=self.softmax(x)
        return x           

In [26]:
mod=MLP()

In [27]:
import torch.optim as optim

In [28]:
opt=optim.SGD(mod.parameters(),lr=0.1)

In [29]:
Loss=[]
num_epoch=10
for i in range(num_epoch):
    for j,batch in enumerate(mnist_batched):
        x=batch['X']
        Y=batch['y']
        p=mod(x.float())
        loss=CE(p,Y)
        loss.backward()
        Loss.append(loss.item())
        acc=(p.argmax(axis=1)==Y).float().mean().item()
        if j%100 == 0:
            print(f"Epoch {i+1}, iter {j+1}, loss {loss.item()}, acc {acc}")
        opt.step()
        opt.zero_grad()

Epoch 1, iter 1, loss 2.4486403465270996, acc 0.14000000059604645
Epoch 1, iter 101, loss 2.1653308868408203, acc 0.3799999952316284
Epoch 1, iter 201, loss 1.9942572116851807, acc 0.41999998688697815
Epoch 1, iter 301, loss 1.8990768194198608, acc 0.38999998569488525


/var/folders/7h/01sjg2sx6tl3y72r2klm24c80000gn/T/ipykernel_73154/2704172253.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.softmax(x)


Epoch 1, iter 401, loss 1.8355108499526978, acc 0.4000000059604645
Epoch 2, iter 1, loss 1.7177814245224, acc 0.4099999964237213
Epoch 2, iter 101, loss 1.6351189613342285, acc 0.3799999952316284
Epoch 2, iter 201, loss 1.5560745000839233, acc 0.46000000834465027
Epoch 2, iter 301, loss 1.5303338766098022, acc 0.5099999904632568
Epoch 2, iter 401, loss 1.533867359161377, acc 0.5400000214576721
Epoch 3, iter 1, loss 1.4266786575317383, acc 0.4699999988079071
Epoch 3, iter 101, loss 1.3606985807418823, acc 0.49000000953674316
Epoch 3, iter 201, loss 1.3400490283966064, acc 0.47999998927116394
Epoch 3, iter 301, loss 1.3406490087509155, acc 0.5099999904632568
Epoch 3, iter 401, loss 1.3999136686325073, acc 0.550000011920929
Epoch 4, iter 1, loss 1.3073148727416992, acc 0.5099999904632568
Epoch 4, iter 101, loss 1.2108964920043945, acc 0.5199999809265137
Epoch 4, iter 201, loss 1.2243218421936035, acc 0.5
Epoch 4, iter 301, loss 1.235899806022644, acc 0.5400000214576721
Epoch 4, iter 401, 

In [30]:
## We can make one last change, instead of defining a loss function ourselves we will use a predifined one

In [31]:
criterion=torch.nn.CrossEntropyLoss()

In [32]:
mod=MLP()
opt=optim.SGD(mod.parameters(),lr=0.1)
Loss=[]
num_epoch=10
for i in range(num_epoch):
    for j,batch in enumerate(mnist_batched):
        x=batch['X']
        Y=batch['y']
        p=mod(x.float())
        loss=criterion(p,Y.long())
        loss.backward()
        Loss.append(loss.item())
        acc=(p.argmax(axis=1)==Y).float().mean().item()
        if j%100 == 0:
            print(f"Epoch {i+1}, iter {j+1}, loss {loss.item()}, acc {acc}")
        opt.step()
        opt.zero_grad()

Epoch 1, iter 1, loss 2.3066108226776123, acc 0.12999999523162842
Epoch 1, iter 101, loss 2.3004531860351562, acc 0.1899999976158142
Epoch 1, iter 201, loss 2.287522792816162, acc 0.25
Epoch 1, iter 301, loss 2.2897162437438965, acc 0.20999999344348907


/var/folders/7h/01sjg2sx6tl3y72r2klm24c80000gn/T/ipykernel_73154/2704172253.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.softmax(x)


Epoch 1, iter 401, loss 2.298501968383789, acc 0.14000000059604645
Epoch 2, iter 1, loss 2.293881416320801, acc 0.14000000059604645
Epoch 2, iter 101, loss 2.2832610607147217, acc 0.20999999344348907
Epoch 2, iter 201, loss 2.2658064365386963, acc 0.2800000011920929
Epoch 2, iter 301, loss 2.2735707759857178, acc 0.17000000178813934
Epoch 2, iter 401, loss 2.2860660552978516, acc 0.14000000059604645
Epoch 3, iter 1, loss 2.278973340988159, acc 0.18000000715255737
Epoch 3, iter 101, loss 2.2609593868255615, acc 0.2199999988079071
Epoch 3, iter 201, loss 2.2315773963928223, acc 0.27000001072883606
Epoch 3, iter 301, loss 2.252167224884033, acc 0.17000000178813934
Epoch 3, iter 401, loss 2.270395278930664, acc 0.17000000178813934
Epoch 4, iter 1, loss 2.258037805557251, acc 0.18000000715255737
Epoch 4, iter 101, loss 2.232966184616089, acc 0.23000000417232513
Epoch 4, iter 201, loss 2.1896190643310547, acc 0.3100000023841858
Epoch 4, iter 301, loss 2.227919578552246, acc 0.200000002980232

### Using keras and tensorflow to build the neural networks

- Using tensors
- Using Keras low level api
- Using Keras Functional api
- Using Keras Sequential api

In [33]:
import tensorflow as tf

### Using tensors

In [34]:
X=mnist_train.drop('label',axis=1).values/255.0
y=mnist_train['label'].values
X = tf.constant(X,dtype='float32')
y = tf.constant(y,dtype='float32')

Metal device set to: Apple M1


2021-12-30 18:03:34.406042: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-30 18:03:34.406161: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [35]:
def make_bias(k, initializer):
    return tf.Variable(initializer(shape=[k,], dtype=tf.float32))
def make_weights(n,k,initializer):
    return tf.Variable(initializer(shape=[n,k], dtype=tf.float32))          

In [36]:
w1 = make_weights(784,3,tf.random_normal_initializer())
b1 = make_bias(3,tf.random_normal_initializer())
w2 = make_weights(3,10,tf.random_normal_initializer())
b2 = make_bias(10,tf.random_normal_initializer())

In [37]:
def network(X,w1,b1,w2,b2):
    z1=tf.matmul(X,w1)+b1
    res1=tf.math.sigmoid(z1)
    z2=tf.matmul(res1,w2)+b2
    probs=tf.nn.softmax(z2,axis=1)
    return probs
CE = tf.keras.losses.SparseCategoricalCrossentropy()

In [38]:
## Forward pass
p=network(X,w1,b1,w2,b2)

In [39]:
CE(y,p)

<tf.Tensor: shape=(), dtype=float32, numpy=2.3035123>

In [40]:
## training loop
X=mnist_train.drop('label',axis=1).values/255.0
y=mnist_train['label'].values
X = tf.constant(X,dtype='float32')
y = tf.constant(y,dtype='float32')

w1 = make_weights(784,3,tf.zeros_initializer())
b1 = make_bias(3,tf.zeros_initializer())
w2 = make_weights(3,10,tf.zeros_initializer())
b2 = make_bias(10,tf.zeros_initializer())

lr=0.1
Loss=[]
for i in range(10):
    with tf.GradientTape() as tape:
        p=network(X,w1,b1,w2,b2)
        loss = CE(y,p)
    pred = tf.cast(tf.argmax(p,axis=1),tf.int32)
    mask = tf.equal(pred,tf.cast(y,dtype = tf.int32))
    mask = tf.cast(mask,dtype=tf.int32)
    acc = tf.reduce_mean(mask)
    print(f"Iter {i+1}, loss {loss.numpy()}, acc {acc.numpy()}")
    ## update the weights
    gw1,gb1,gw2,gb2 = tape.gradient(loss,[w1,b1,w2,b2])
    w1.assign_sub(lr*gw1)
    b1.assign_sub(lr*gb1)
    w2.assign_sub(lr*gw2)
    b2.assign_sub(lr*b2)

Iter 1, loss 2.3025851249694824, acc 0
Iter 2, loss 2.3025636672973633, acc 0
Iter 3, loss 2.3025424480438232, acc 0
Iter 4, loss 2.3025214672088623, acc 0
Iter 5, loss 2.3025012016296387, acc 0
Iter 6, loss 2.302480697631836, acc 0
Iter 7, loss 2.3024609088897705, acc 0
Iter 8, loss 2.302441120147705, acc 0
Iter 9, loss 2.302421808242798, acc 0
Iter 10, loss 2.3024024963378906, acc 0


### Using keras-low-level api

In [41]:
class Linear(tf.keras.layers.Layer):
    """y = w.x + b"""

    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [42]:
class Model(tf.keras.layers.Layer):
    def __init__(self,layer_1_shape,layer_1_num_units, layer_2_num_units,layer_2_shape):
        super(Model, self).__init__()
        self.layer1 = Linear(units = layer_1_num_units,input_dim=layer_1_shape)
        self.layer2 = Linear(units=layer_2_num_units,input_dim=layer_2_shape)
    
    def call(self,inputs):
        x = self.layer1(inputs)
        x = tf.math.sigmoid(x)
        x = self.layer2(x)
        return tf.nn.softmax(x,axis=1)

In [43]:
model = Model(784,3,10,3)

In [44]:
X=mnist_train.drop('label',axis=1).values/255.0
y=mnist_train['label'].values
X = tf.constant(X,dtype='float32')
y = tf.constant(y,dtype='float32')
mnist_data = tf.data.Dataset.from_tensor_slices((X,y))

In [45]:
mnist_data = mnist_data.shuffle(buffer_size=1024).batch(64)

In [46]:
# Loss and optimizer.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

In [47]:
for step, (x, y_1) in enumerate(mnist_data):
    with tf.GradientTape() as tape:

        # Forward pass.
        probs = model(x)
        pred = tf.cast(tf.argmax(probs,axis=1),tf.int32)
        mask = tf.equal(pred,tf.cast(y_1,dtype=tf.int32))
        mask = tf.cast(mask,dtype=tf.int32)
        acc = tf.reduce_mean(mask)


        # External loss value for this batch.
        loss = loss_fn(y_1, probs)

        # Add the losses created during the forward pass.
        loss += sum(model.losses)

        # Get gradients of the loss wrt the weights.
        gradients = tape.gradient(loss, model.trainable_weights)

    # Update the weights of our linear layer.
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))

    # Logging.
    if step % 100 == 0:
        print("Step:", step, "Loss:", float(loss),'Accuracy:',float(acc))

Step: 0 Loss: 2.2941694259643555 Accuracy: 0.0
Step: 100 Loss: 2.3048853874206543 Accuracy: 0.0
Step: 200 Loss: 2.3071470260620117 Accuracy: 0.0
Step: 300 Loss: 2.3042125701904297 Accuracy: 0.0
Step: 400 Loss: 2.3023064136505127 Accuracy: 0.0
Step: 500 Loss: 2.2969107627868652 Accuracy: 0.0
Step: 600 Loss: 2.3000707626342773 Accuracy: 0.0


### Using keras functional api

In [48]:
inputs = tf.keras.Input(shape=(784,))
x=tf.keras.layers.Dense(units=3,activation="sigmoid")(inputs)
output = tf.keras.layers.Dense(units=10,activation="softmax")(x)
model = tf.keras.Model(inputs,output)

In [49]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 3)                 2355      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                40        
Total params: 2,395
Trainable params: 2,395
Non-trainable params: 0
_________________________________________________________________


In [50]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [51]:
model.fit(mnist_data,epochs=10)

Epoch 1/10
 25/657 [>.............................] - ETA: 2s - loss: 2.3642 - accuracy: 0.1075

2021-12-30 18:03:41.310012: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-30 18:03:41.310085: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


657/657 [==============================] - 3s 4ms/step - loss: 2.2477 - accuracy: 0.1812
Epoch 2/10
657/657 [==============================] - 3s 4ms/step - loss: 2.1109 - accuracy: 0.3613
Epoch 3/10
657/657 [==============================] - 2s 3ms/step - loss: 2.0013 - accuracy: 0.4408
Epoch 4/10
657/657 [==============================] - 2s 4ms/step - loss: 1.9027 - accuracy: 0.4427
Epoch 5/10
657/657 [==============================] - 2s 4ms/step - loss: 1.8149 - accuracy: 0.4469
Epoch 6/10
657/657 [==============================] - 3s 4ms/step - loss: 1.7375 - accuracy: 0.4690
Epoch 7/10
657/657 [==============================] - 3s 4ms/step - loss: 1.6692 - accuracy: 0.4936
Epoch 8/10
657/657 [==============================] - 3s 4ms/step - loss: 1.6089 - accuracy: 0.5139
Epoch 9/10
657/657 [==============================] - 3s 4ms/step - loss: 1.5554 - accuracy: 0.5257
Epoch 10/10
657/657 [==============================] - 3s 4ms/step - loss: 1.5077 - accuracy: 0.5354


### Keras Sequential api

In [52]:
model = tf.keras.Sequential(
    [
    tf.keras.layers.Dense(units=3,activation='sigmoid',input_shape=(784,)),
    tf.keras.layers.Dense(units=10,activation='softmax')
    ]
)

In [53]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [54]:
model.fit(mnist_data,epochs=10)

Epoch 1/10
 28/657 [>.............................] - ETA: 2s - loss: 2.3587 - accuracy: 0.1144

2021-12-30 18:04:07.504861: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


657/657 [==============================] - 3s 4ms/step - loss: 2.2477 - accuracy: 0.1812
Epoch 2/10
657/657 [==============================] - 3s 4ms/step - loss: 2.1110 - accuracy: 0.3611
Epoch 3/10
657/657 [==============================] - 3s 5ms/step - loss: 2.0014 - accuracy: 0.4416
Epoch 4/10
657/657 [==============================] - 2s 3ms/step - loss: 1.9028 - accuracy: 0.4433
Epoch 5/10
657/657 [==============================] - 2s 3ms/step - loss: 1.8149 - accuracy: 0.4473
Epoch 6/10
657/657 [==============================] - 2s 3ms/step - loss: 1.7375 - accuracy: 0.4693: 0s
Epoch 7/10
657/657 [==============================] - 2s 3ms/step - loss: 1.6692 - accuracy: 0.4931
Epoch 8/10
657/657 [==============================] - 2s 3ms/step - loss: 1.6089 - accuracy: 0.5132
Epoch 9/10
657/657 [==============================] - 2s 3ms/step - loss: 1.5554 - accuracy: 0.5258
Epoch 10/10
657/657 [==============================] - 2s 3ms/step - loss: 1.5077 - accuracy: 0.5354
